
# STRUCTURE

In [1]:
import ipyrad.analysis as ipa
import toyplot
import ipyparallel as ipp

In [2]:
## look for running ipcluster instance, and create load-balancer
ipyclient = ipp.Client()
print("{} engines found".format(len(ipyclient)))

24 engines found


In [3]:
# the path to your HDF5 formatted snps file
data = "./G_cy_5rm_v9.snps.hdf5"

In [12]:
imap = {
#"ref": ["reference"],
"Inam": ["G_cy_T12385_In","G_cy_T310_In","G_cy_T23196_In","G_cy_T7636_In"],
"Puru": ["G_cy_82508_pu","G_cy_T12275_pu","G_cy_T12279_pu","G_cy_T12392_pu","G_cy_T13184_pu","G_cy_T26228_pu","G_cy_T26229_pu","G_cy_T26252_pu"],
"JiGu": ["G_cy_T3343_jigu","G_cy_T3384_jigu","G_cy_T3385_jigu"],
"Mach": ["G_cy_J296_ma","G_cy_J477_ma","G_cy_J773_ma","G_cy_T13251_ma","G_cy_T363_ma","G_cy_T364_ma"],
"Roar": ["G_cy_J691_roar","G_cy_J694_roar"],
"ArSu": ["G_cy_80582_arsu","G_cy_85678_arsu","G_cy_80701_arsu","G_cy_80801_arsu","G_cy_80826_arsu","G_cy_81108_arsu","G_cy_81118_arsu","G_cy_85499_arsu"],
"SuTa": ["G_cy_85356_suta","G_cy_86297_suta","G_cy_86321_suta","G_cy_86458_suta","G_cy_86478_suta","G_cy_T14558_suta","G_cy_T16693_suta","G_cy_T18563_suta","G_cy_T18620_suta"],
"Para": ["G_cy_T10897_pa","G_cy_T11062_pa","G_cy_T16771_pa","G_cy_T1705_pa","G_cy_T18744_pa","G_cy_T19429_pa","G_cy_T19520_pa","G_cy_T19765_pa","G_cy_T2497_pa","G_cy_T6579_pa","G_cy_T9133_pa"]
}

# minimum % of samples that must be present in each SNP from each group
minmap = {i: 0.5 for i in imap}

In [12]:
# init analysis object with input data and (optional) parameter options
struct = ipa.structure(
    name="G_cy_str",
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.95,
)

Samples: 52
Sites before filtering: 1130537
Filtered (indels): 0
Filtered (bi-allel): 21079
Filtered (mincov): 1079809
Filtered (minmap): 820258
Filtered (combined): 1081015
Sites after filtering: 49522
Sites containing missing values: 42350 (85.52%)
Missing values in SNP matrix: 67609 (2.63%)


In [14]:
struct.mainparams.burnin = 50000
struct.mainparams.numreps = 250000
struct.write_structure_files(abs)

('/array1/lmusher/rio_roosevelt_outfiles/G_cy_5rm_v9_outfiles/analysis-structure/tmp-G_cy_str2-<built-in function abs>-1.mainparams.txt',
 '/array1/lmusher/rio_roosevelt_outfiles/G_cy_5rm_v9_outfiles/analysis-structure/tmp-G_cy_str2-<built-in function abs>-1.extraparams.txt',
 '/array1/lmusher/rio_roosevelt_outfiles/G_cy_5rm_v9_outfiles/analysis-structure/tmp-G_cy_str2-<built-in function abs>-1.strfile.txt')

In [ ]:
struct.run(nreps=10, kpop=[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], ipyclient=ipyclient, force=True)

Parallel connection | amnh-gen-001.internal.amnh.org: 20 cores
[############        ]  63% 1 day, 2:48:18 | running 110 structure jobs 

In [48]:
rerun = ipa.structure(
    data=data, 
    name="G_cy_str", 
    workdir="analysis-structure",
    imap=imap,
    load_only=True,
)

110 previous results loaded for run [G_cy_str]


In [49]:
etable = struct.get_evanno_table([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
etable

,Nreps,lnPK,lnPPK,deltaK,estLnProbMean,estLnProbStdev
2,10,0.00,0.00,0.000000,-63099.88,690.813760
3,10,6839.85,8521.36,10.627458,-56260.03,801.824830
4,10,-1681.51,2813.03,0.876252,-57941.54,3210.297857
5,10,1131.52,270548.89,90.331776,-56810.02,2995.057803
6,10,-269417.37,466603.32,0.813844,-326227.39,573332.704979
7,10,197185.95,385838.61,1.791553,-129041.44,215365.417458
8,10,-188652.66,82862.37,0.240171,-317694.10,345013.837754
9,10,-105790.29,136660.62,0.405655,-423484.39,336888.570210
10,10,30870.33,132281.79,0.427659,-392614.06,309316.003239
11,10,-101411.46,263784.50,0.552141,-494025.52,477748.659139


In [50]:
# get canvas object and set size
canvas = toyplot.Canvas(width=400, height=300)

# plot the mean log probability of the models in red
axes = canvas.cartesian(ylabel="estLnProbMean")
axes.plot(etable.estLnProbMean * -1, color="darkred", marker="o")
axes.y.spine.style = {"stroke": "darkred"}

# plot delta K with its own scale bar of left side and in blue
axes = axes.share("x", ylabel="deltaK", ymax=etable.deltaK.max() + etable.deltaK.max() * .25)
axes.plot(etable.deltaK, color="steelblue", marker="o");
axes.y.spine.style = {"stroke": "steelblue"}

# set x labels
axes.x.ticks.locator = toyplot.locator.Explicit(range(len(etable.index)), etable.index)
axes.x.label.text = "K (N ancestral populations)"

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t7527bc65c1364ce88d1961685a44dd60" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 2 3 4 5 6 7 8 9 10 11 12 K (N ancestral populations) 0 100000 200000 300000 400000 500000 estLnProbMean 0 40 80 120 deltaK

In [51]:
k = 2
table = struct.get_clumpp_table(k)

[K2] 10/10 results permuted across replicates (max_var=0).


In [52]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [53]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t620f3520bf45425796b54b65b5adc6f3" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> G_cy_T12385_In G_cy_T310_In G_cy_T23196_In G_cy_T7636_In G_cy_82508_pu G_cy_T12275_pu G_cy_T12279_pu G_cy_T12392_pu G_cy_T13184_pu G_cy_T26228_pu G_cy_T26229_pu G_cy_T26252_pu G_cy_T3343_jigu G_cy_T3384_jigu G_cy_T3385_jigu G_cy_J296_ma G_cy_J477_ma G_cy_J773_ma G_cy_T13251_ma G_cy_T363_ma G_cy_T364_ma G_cy_J691_roar G_cy_J694_roar G_cy_80582_arsu G_cy_85678_arsu G_cy_80701_arsu G_cy_80801_arsu G_cy_80826_arsu G_cy_81108_arsu G_cy_81118_arsu G_cy_85499_arsu G_cy_85356_suta G_cy_86297_suta G_cy_86321_suta G_cy_86458_suta G_cy_86478_suta G_cy_T14558_suta G_cy_T16693_suta G_cy_T18563_suta G_cy_T18620_suta G_cy_T10897_pa G_cy_T11062_pa G_cy_T16771_pa G_cy_T1705_pa G_cy_T18744_pa G_cy_T19429_pa G_cy_T19520_pa G_cy_T19765_pa G_cy_T2497_pa G_cy_T6579_pa G_cy_T9133_pa 0.0 0.5 1.0

In [54]:
k = 3
table = struct.get_clumpp_table(k)

[K3] 10/10 results permuted across replicates (max_var=0).


In [55]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [56]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t3c1146b736e0453bb99dbb12de46339f" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> G_cy_T12385_In G_cy_T310_In G_cy_T23196_In G_cy_T7636_In G_cy_82508_pu G_cy_T12275_pu G_cy_T12279_pu G_cy_T12392_pu G_cy_T13184_pu G_cy_T26228_pu G_cy_T26229_pu G_cy_T26252_pu G_cy_T3343_jigu G_cy_T3384_jigu G_cy_T3385_jigu G_cy_J296_ma G_cy_J477_ma G_cy_J773_ma G_cy_T13251_ma G_cy_T363_ma G_cy_T364_ma G_cy_J691_roar G_cy_J694_roar G_cy_80582_arsu G_cy_85678_arsu G_cy_80701_arsu G_cy_80801_arsu G_cy_80826_arsu G_cy_81108_arsu G_cy_81118_arsu G_cy_85499_arsu G_cy_85356_suta G_cy_86297_suta G_cy_86321_suta G_cy_86458_suta G_cy_86478_suta G_cy_T14558_suta G_cy_T16693_suta G_cy_T18563_suta G_cy_T18620_suta G_cy_T10897_pa G_cy_T11062_pa G_cy_T16771_pa G_cy_T1705_pa G_cy_T18744_pa G_cy_T19429_pa G_cy_T19520_pa G_cy_T19765_pa G_cy_T2497_pa G_cy_T6579_pa G_cy_T9133_pa 0.0 0.5 1.0

In [57]:
k = 4
table = struct.get_clumpp_table(k)

[K4] 10/10 results permuted across replicates (max_var=0).


In [58]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [59]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t198b7775712548aab32fbb0b9364073f" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> G_cy_T12385_In G_cy_T310_In G_cy_T23196_In G_cy_T7636_In G_cy_82508_pu G_cy_T12275_pu G_cy_T12279_pu G_cy_T12392_pu G_cy_T13184_pu G_cy_T26228_pu G_cy_T26229_pu G_cy_T26252_pu G_cy_T3343_jigu G_cy_T3384_jigu G_cy_T3385_jigu G_cy_J296_ma G_cy_J477_ma G_cy_J773_ma G_cy_T13251_ma G_cy_T363_ma G_cy_T364_ma G_cy_J691_roar G_cy_J694_roar G_cy_80582_arsu G_cy_85678_arsu G_cy_80701_arsu G_cy_80801_arsu G_cy_80826_arsu G_cy_81108_arsu G_cy_81118_arsu G_cy_85499_arsu G_cy_85356_suta G_cy_86297_suta G_cy_86321_suta G_cy_86458_suta G_cy_86478_suta G_cy_T14558_suta G_cy_T16693_suta G_cy_T18563_suta G_cy_T18620_suta G_cy_T10897_pa G_cy_T11062_pa G_cy_T16771_pa G_cy_T1705_pa G_cy_T18744_pa G_cy_T19429_pa G_cy_T19520_pa G_cy_T19765_pa G_cy_T2497_pa G_cy_T6579_pa G_cy_T9133_pa 0.0 0.5 1.0

In [60]:
k = 5
table = struct.get_clumpp_table(k)

[K5] 10/10 results permuted across replicates (max_var=0).


In [61]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [62]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="te1ef5e5b5a60470abacec89564126372" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> G_cy_T12385_In G_cy_T310_In G_cy_T23196_In G_cy_T7636_In G_cy_82508_pu G_cy_T12275_pu G_cy_T12279_pu G_cy_T12392_pu G_cy_T13184_pu G_cy_T26228_pu G_cy_T26229_pu G_cy_T26252_pu G_cy_T3343_jigu G_cy_T3384_jigu G_cy_T3385_jigu G_cy_J296_ma G_cy_J477_ma G_cy_J773_ma G_cy_T13251_ma G_cy_T363_ma G_cy_T364_ma G_cy_J691_roar G_cy_J694_roar G_cy_80582_arsu G_cy_85678_arsu G_cy_80701_arsu G_cy_80801_arsu G_cy_80826_arsu G_cy_81108_arsu G_cy_81118_arsu G_cy_85499_arsu G_cy_85356_suta G_cy_86297_suta G_cy_86321_suta G_cy_86458_suta G_cy_86478_suta G_cy_T14558_suta G_cy_T16693_suta G_cy_T18563_suta G_cy_T18620_suta G_cy_T10897_pa G_cy_T11062_pa G_cy_T16771_pa G_cy_T1705_pa G_cy_T18744_pa G_cy_T19429_pa G_cy_T19520_pa G_cy_T19765_pa G_cy_T2497_pa G_cy_T6579_pa G_cy_T9133_pa 0.0 0.5 1.0

In [63]:
k = 6
table = struct.get_clumpp_table(k)

[K6] 10/10 results permuted across replicates (max_var=0).


In [64]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [65]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="tae819ff3ba1d4a95832fb5365fed2de0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> G_cy_T12385_In G_cy_T310_In G_cy_T23196_In G_cy_T7636_In G_cy_82508_pu G_cy_T12275_pu G_cy_T12279_pu G_cy_T12392_pu G_cy_T13184_pu G_cy_T26228_pu G_cy_T26229_pu G_cy_T26252_pu G_cy_T3343_jigu G_cy_T3384_jigu G_cy_T3385_jigu G_cy_J296_ma G_cy_J477_ma G_cy_J773_ma G_cy_T13251_ma G_cy_T363_ma G_cy_T364_ma G_cy_J691_roar G_cy_J694_roar G_cy_80582_arsu G_cy_85678_arsu G_cy_80701_arsu G_cy_80801_arsu G_cy_80826_arsu G_cy_81108_arsu G_cy_81118_arsu G_cy_85499_arsu G_cy_85356_suta G_cy_86297_suta G_cy_86321_suta G_cy_86458_suta G_cy_86478_suta G_cy_T14558_suta G_cy_T16693_suta G_cy_T18563_suta G_cy_T18620_suta G_cy_T10897_pa G_cy_T11062_pa G_cy_T16771_pa G_cy_T1705_pa G_cy_T18744_pa G_cy_T19429_pa G_cy_T19520_pa G_cy_T19765_pa G_cy_T2497_pa G_cy_T6579_pa G_cy_T9133_pa 0.0 0.5 1.0

# PCA

In [66]:
#!conda install scikit-learn -c conda-forge -y

In [13]:
import ipyrad.analysis as ipa
import pandas as pd
import toyplot

In [14]:
# init pca object with input data and (optional) parameter options
pca = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method="sample",
)

Samples: 51
Sites before filtering: 1130537
Filtered (indels): 0
Filtered (bi-allel): 21079
Filtered (mincov): 625064
Filtered (minmap): 1123199
Filtered (combined): 1123365
Sites after filtering: 7172
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)


In [91]:
# run the PCA analysis
pca.run()

Subsampling SNPs: 30391/302918


In [92]:
pca.draw()

(<toyplot.canvas.Canvas at 0x2b0f07661da0>,
 <toyplot.mark.Point at 0x2b0f6fe8a940>)

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t812a40a5cac740d19830259d18d23ad1" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> G_cy_80582_arsu G_cy_80701_arsu G_cy_80801_arsu G_cy_80826_arsu G_cy_81108_arsu G_cy_81118_arsu G_cy_82508_pu G_cy_85356_suta G_cy_85499_arsu G_cy_85678_arsu G_cy_86297_suta G_cy_86321_suta G_cy_86458_suta G_cy_86478_suta G_cy_J296_ma G_cy_J477_ma G_cy_J691_roar G_cy_J694_roar G_cy_J773_ma G_cy_T10897_pa G_cy_T11062_pa G_cy_T12275_pu G_cy_T12279_pu G_cy_T12385_In G_cy_T12392_pu G_cy_T13184_pu G_cy_T13251_ma G_cy_T14558_suta G_cy_T16693_suta G_cy_T16771_pa G_cy_T1705_pa G_cy_T18563_suta G_cy_T18620_suta G_cy_T18744_pa G_cy_T19429_pa G_cy_T19520_pa G_cy_T19765_pa G_cy_T23196_In G_cy_T2497_pa G_cy_T26228_pu G_cy_T26229_pu G_cy_T26252_pu G_cy_T310_In G_cy_T3343_jigu G_cy_T3384_jigu G_cy_T3385_jigu G_cy_T363_ma G_cy_T364_ma G_cy_T6579_pa G_cy_T7636_In G_cy_T9133_pa -20 -10 0 10 20 30 PC0 (18.5%) explained -30 0 30 PC1 (16.0%) explained Inam Puru JiGu Mach Roar ArSu SuTa Para

In [93]:
# store the PC axes as a dataframe
df = pd.DataFrame(pca.pcaxes[0], index=pca.names)

# write the PC axes to a CSV file
df.to_csv("G_cy_pca_analysis.csv")

# show the first ten samples and the first 10 PC axes
df.iloc[:10, :10].round(2)

,0,1,2,3,4,5,6,7,8,9
G_cy_80582_arsu,30.82,0.79,-0.93,0.55,-0.12,-0.79,-1.49,-7.54,-0.50,3.28
G_cy_80701_arsu,30.42,0.15,-0.59,0.29,-0.46,0.29,0.18,-7.67,-4.21,-2.83
G_cy_80801_arsu,29.65,0.23,-1.14,0.51,-1.07,1.15,-2.55,-9.20,-1.55,2.52
G_cy_80826_arsu,29.73,0.40,-0.99,0.99,-0.73,-2.10,-0.22,-12.32,-3.90,-3.19
G_cy_81108_arsu,29.63,0.88,-0.59,0.47,-1.84,-0.25,-1.93,-10.94,-2.99,-1.52
G_cy_81118_arsu,30.21,1.39,-0.68,0.54,0.50,-0.12,1.40,-11.50,-0.98,0.78
G_cy_82508_pu,-17.85,21.21,2.56,25.60,24.68,0.58,0.41,-0.95,1.01,-0.29
G_cy_85356_suta,30.41,1.09,-1.22,1.66,1.30,0.32,1.04,1.91,-1.53,-2.47
G_cy_85499_arsu,30.84,0.96,-0.58,0.93,0.52,-0.87,-1.04,-14.48,-4.43,0.32
G_cy_85678_arsu,30.79,0.57,-0.73,0.49,1.62,1.02,-0.08,-10.19,-2.62,0.83


# Subsampling with replication

In [72]:
# plot PC axes 0 and 2 with many replicate subsamples
pca.run(nreplicates=25, seed=12345)
pca.draw(0, 2);

Subsampling SNPs: 30391/302918


<svg class="toyplot-canvas-Canvas" height="300.0px" id="t221282d32c3342a5b774a6b60176287a" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <g class="toyplot-Datum" style="fill:rgb(62%,0.4%,25.9%);fill-opacity:0.036000000000000004;opacity:1.0;stroke:none" transform="translate(66.94611772656985, 234.12125403765

# Kmeans imputation (integer)

In [73]:
# kmeans imputation 
pca3 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method=7,
)

# run and draw results for kmeans clustering into 7 groups
pca3.run(nreplicates=25, seed=123)
pca3.draw(0, 2);

Kmeans clustering: iter=0, K=7, mincov=0.9, minmap={'global': 0.5}
Samples: 51
Sites before filtering: 1130537
Filtered (indels): 0
Filtered (bi-allel): 21079
Filtered (mincov): 990529
Filtered (minmap): 625064
Filtered (combined): 993791
Sites after filtering: 136746
Sites containing missing values: 129574 (94.76%)
Missing values in SNP matrix: 416173 (5.97%)
Imputation: 'sampled'; (0, 1, 2) = 58.1%, 6.1%, 35.8%
{0: ['G_cy_J296_ma', 'G_cy_J477_ma', 'G_cy_J691_roar', 'G_cy_J694_roar', 'G_cy_J773_ma', 'G_cy_T13251_ma', 'G_cy_T363_ma', 'G_cy_T364_ma'], 1: ['G_cy_T12275_pu', 'G_cy_T12279_pu', 'G_cy_T12392_pu', 'G_cy_T13184_pu', 'G_cy_T26228_pu'], 2: ['G_cy_80582_arsu', 'G_cy_80701_arsu', 'G_cy_80801_arsu', 'G_cy_80826_arsu', 'G_cy_81108_arsu', 'G_cy_81118_arsu', 'G_cy_85356_suta', 'G_cy_85499_arsu', 'G_cy_85678_arsu', 'G_cy_86297_suta', 'G_cy_86321_suta', 'G_cy_86458_suta', 'G_cy_86478_suta', 'G_cy_T14558_suta', 'G_cy_T16693_suta', 'G_cy_T18563_suta', 'G_cy_T18620_suta'], 3: ['G_cy_T10897

<svg class="toyplot-canvas-Canvas" height="300.0px" id="teda68c49f09f4320985ba25bcf54cf15" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <circ

In [74]:
import toyplot.pdf

# save returned plot objects as variables
canvas, axes, mark = pca3.draw(0, 2)

# pass the canvas object to toyplot render function
toyplot.pdf.render(canvas, "G_cy_PCA-kmeans-7.pdf")

# T-SNE (ASSESSING COMPONENT LOADINGS)

In [7]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE

In [21]:
pca.run_tsne(subsample=True, perplexity=8.0, n_iter=1000000, seed=123)

Subsampling SNPs: 1079/7172


In [22]:
pca.draw(size=8);

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tb54ad24095cd4598a3d2a118ac294e9f" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> G_cy_80582_arsu G_cy_80701_arsu G_cy_80801_arsu G_cy_80826_arsu G_cy_81108_arsu G_cy_81118_arsu G_cy_82508_pu G_cy_85356_suta G_cy_85499_arsu G_cy_85678_arsu G_cy_86297_suta G_cy_86321_suta G_cy_86458_suta G_cy_86478_suta G_cy_J296_ma G_cy_J477_ma G_cy_J691_roar G_cy_J694_roar G_cy_J773_ma G_cy_T10897_pa G_cy_T11062_pa G_cy_T12275_pu G_cy_T12279_pu G_cy_T12385_In G_cy_T12392_pu G_cy_T13184_pu G_cy_T13251_ma G_cy_T14558_suta G_cy_T16693_suta G_cy_T16771_pa G_cy_T1705_pa G_cy_T18563_suta G_cy_T18620_suta G_cy_T18744_pa G_cy_T19429_pa G_cy_T19520_pa G_cy_T19765_pa G_cy_T23196_In G_cy_T2497_pa G_cy_T26228_pu G_cy_T26229_pu G_cy_T26252_pu G_cy_T310_In G_cy_T3343_jigu G_cy_T3384_jigu G_cy_T3385_jigu G_cy_T363_ma G_cy_T364_ma G_cy_T6579_pa G_cy_T7636_In G_cy_T9133_pa -40 0 40 80 TNSE component 1 -100 0 100 TNSE component 2 Para Puru Roar ArSu Inam SuTa JiGu Mach

In [23]:
# store the PC axes as a dataframe
df = pd.DataFrame(pca.pcaxes[0], index=pca.names)

# write the PC axes to a CSV file
df.to_csv("G_cy_TSNE.csv")

# show the first ten samples and the first 10 PC axes
df.head()

,0,1
G_cy_80582_arsu,-29.968,-93.493
G_cy_80701_arsu,-35.568,-77.026
G_cy_80801_arsu,-32.888,-100.457
G_cy_80826_arsu,-37.213,-94.466
G_cy_81108_arsu,-42.121,-79.816


In [97]:
import toyplot.pdf

# save returned plot objects as variables
canvas, axes, mark = pca.draw(size=8)

# pass the canvas object to toyplot render function
toyplot.pdf.render(canvas, "G_cy_TSNE_perp8.pdf")

# GENETIC DISTNANCES

In [79]:
import ipyrad.analysis as ipa
import toyplot

In [80]:
# load the snp data into distance tool with arguments
from ipyrad.analysis.distance import Distance
dist = Distance(
    data=data, 
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method="sample",
    subsample_snps=False,
)
dist.run()

Samples: 51
Sites before filtering: 1130537
Filtered (indels): 0
Filtered (bi-allel): 21079
Filtered (mincov): 625064
Filtered (minmap): 820258
Filtered (combined): 827619
Sites after filtering: 302918
Sites containing missing values: 295746 (97.63%)
Missing values in SNP matrix: 2062023 (13.35%)
Imputation: 'sampled'; (0, 1, 2) = 56.5%, 4.5%, 39.0%


In [81]:
# save to a CSV file
dist.dists.to_csv("G_cy_distances.csv")

In [82]:
# save to a CSV file with no labels (eems style)
dist.dists.to_csv(
    "G_cy_distances_eems.csv",
    header=None,
    index=False,
    sep=" ",
)

In [83]:
# get list of concatenated names from each group
ordered_names = []
for group in dist.imap.values():
    ordered_names += group

# reorder matrix to match name order    
ordered_matrix = dist.dists[ordered_names].T[ordered_names]

In [84]:
toyplot.matrix(
    ordered_matrix,
    bshow=False,
    tshow=False,
    rlocator=toyplot.locator.Explicit(
        range(len(ordered_names)),
        ordered_names,
));

<svg class="toyplot-canvas-Canvas" height="600px" id="t61f3b1b508cf48a089d9d1343d572711" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 600" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 0.000000 0.113810 0.128233 0.122155 0.160010 0.166213 0.164391 0.165847 0.164513 0.163414 0.163553 0.158779 0.128893 0.129580 0.127850 0.203530 0.205438 0.205339 0.204501 0.204369 0.205798 0.203520 0.211232 0.212807 0.212794 0.212949 0.213091 0.213794 0.212856 0.211298 0.212285 0.211704 0.212434 0.213635 0.212269 0.214051 0.211595 0.215276 0.212619 0.213167 0.215273 0.214976 0.220020 0.223658 0.214844 0.215986 0.215996 0.215296 0.216623 0.216983 0.215633 G_cy_T12385_In 1 0.113810 0.000000 0.120808 0.117583 0.163176 0.170792 0.167015 0.169475 0.167758 0.167359 0.166943 0.162354 0.135459 0.136615 0.133742 0.206756 0.208783 0.209238 0.207911 0.207706 0.209096 0.206858 0.213903 0.215260 0.215530 0.215170 0.215464 0.216002 0.214464 0.213718 0.214751 0.214652 0.215606 0.215606 0.213447 0.215586 0.214061 0.216455 0.214959 0.215025 0.218320 0.218102 0.223490 0.227233 0.218227 0.219858 0.219683 0.218759 0.220297 0.220096 0.218594 G_cy_T310_In 2 0.128233 0.120808 0.000000 0.107458 0.174351 0.180316 0.179266 0.180914 0.179138 0.178184 0.177675 0.173535 0.115833 0.113503 0.111601 0.217792 0.218663 0.219554 0.218742 0.219284 0.220007 0.217280 0.224516 0.224189 0.224216 0.223731 0.224143 0.224067 0.223103 0.222496 0.223734 0.222882 0.223962 0.224780 0.222608 0.224239 0.222687 0.224751 0.223770 0.223453 0.229963 0.228755 0.234618 0.237510 0.228999 0.229600 0.229650 0.229049 0.230759 0.230683 0.229234 G_cy_T23196_In 3 0.122155 0.117583 0.107458 0.000000 0.170776 0.177976 0.174780 0.177127 0.175259 0.174707 0.174113 0.170013 0.115130 0.112793 0.111459 0.213596 0.215728 0.216078 0.214976 0.215147 0.216029 0.213355 0.220954 0.221083 0.221222 0.220901 0.221486 0.221707 0.220030 0.219337 0.220568 0.219980 0.220901 0.221027 0.219033 0.220921 0.220367 0.221737 0.220221 0.220928 0.226144 0.225312 0.230719 0.233657 0.224968 0.226612 0.226636 0.226048 0.226715 0.227147 0.225876 G_cy_T7636_In 4 0.160010 0.163176 0.174351 0.170776 0.000000 0.125239 0.123271 0.123948 0.102397 0.099555 0.066312 0.070749 0.172839 0.175354 0.172779 0.173978 0.174453 0.175331 0.174123 0.173648 0.175691 0.173235 0.180735 0.197931 0.197601 0.197123 0.197661 0.198153 0.197605 0.196106 0.195812 0.196453 0.197123 0.198595 0.195538 0.197895 0.196502 0.199305 0.196964 0.197809 0.190071 0.188718 0.193861 0.197981 0.188903 0.189371 0.189718 0.189586 0.191653 0.190731 0.189276 G_cy_82508_pu 5 0.166213 0.170792 0.180316 0.177976 0.125239 0.000000 0.116494 0.117771 0.112172 0.110875 0.110624 0.121663 0.179530 0.181927 0.180593 0.171710 0.172535 0.172918 0.171888 0.172463 0.175008 0.171911 0.180019 0.198311 0.197902 0.197793 0.197961 0.197674 0.196360 0.196122 0.196159 0.197083 0.197344 0.197311 0.196103 0.197964 0.195911 0.198905 0.196756 0.197482 0.188114 0.187130 0.193343 0.197357 0.187982 0.189243 0.188896 0.188183 0.190659 0.189150 0.188308 G_cy_T12275_pu 6 0.164391 0.167015 0.179266 0.174780 0.123271 0.116494 0.000000 0.114741 0.109010 0.107560 0.107930 0.119491 0.176863 0.179511 0.176896 0.170284 0.171802 0.171063 0.171056 0.171353 0.171891 0.169594 0.177253 0.195875 0.195928 0.195680 0.196502 0.196367 0.195713 0.195614 0.194277 0.195129 0.196446 0.197014 0.194320 0.197119 0.193442 0.198430 0.195350 0.196155 0.186681 0.186298 0.191890 0.195469 0.186575 0.187480 0.187338 0.186968 0.188506 0.187295 0.186711 G_cy_T12279_pu 7 0.165847 0.169475 0.180914 0.177127 0.123948 0.117771 0.114741 0.000000 0.110723 0.108105 0.109142 0.118808 0.178075 0.181066 0.179